# Best footbal transactions 2015-2020
> Analyzing player performance to determine the best transactions

- toc: true 
- badges: true
- comments: false
- categories: [jupyter]

In [1]:
#hide
import pandas as pd
import glob
from IPython.display import HTML
def read_csv_folder(folder_path, file_prefix=None, file_col_name=None):
    all_files = glob.glob(folder_path + "/*.csv")

    li = []
    for filename in all_files:
        
        df = pd.read_csv(filename, index_col=None, header=0)
        if file_prefix != None:
            
            if file_col_name == None: file_col_name = 'file'
            
            col_value = filename.strip(folder_path + '/' + 'players_').strip('.csv')
            df.insert(
                loc=0, 
                column=file_col_name, 
                value=col_value)
        
        li.append(df)

    return pd.concat(li, axis=0, ignore_index=True)
from fastdata.core import *
import plotly.express as px

from fuzzywuzzy import fuzz, process
def fuzzy_merge(left, right, left_on, right_on, threshold=90, limit=1):
    s = right[right_on].tolist()
    
    m = left[left_on].apply(lambda x: process.extract(x, s, limit=limit))    
    left['matches'] = m
    
    m2 = left['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    left['matches'] = m2
    
    return left

# Summary

The following article outlines a methodology to identify the best among the top football transactions between 2015 and 2020.

Two different methodologies to define "best" are presented and discussed.
For this exercise, data from the FIFA videogame to evaluate player performance and a list of football transactions by value from Wikipedia are used.

# 1. Load & prepare data

## Load FIFA dataset for all years

**Description:** This dataset from the FIFA video game includes an assessment of player skills in several categories and contextual information (club, left vs. right-footed, etc.).

**Source:** [Kaggle Dataset](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_19.csv)
Original data from [sofifa.com](https://sofifa.com)

In [2]:
all = read_csv_folder(
    folder_path="FIFA_all_years", 
    file_prefix="players_", 
    file_col_name="year")

In [3]:
all = all.astype(
    dtype={"year" : "int64"})

Set full year number

In [4]:
all["year"] = all["year"].add(
    other=2000)

In [5]:
#hide_input
all.head()

,year,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,2021,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,33,1987-06-24,170,72,Argentina,...,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3
1,2021,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,35,1985-02-05,187,83,Portugal,...,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3
2,2021,200389,https://sofifa.com/player/200389/jan-oblak/210002,J. Oblak,Jan Oblak,27,1993-01-07,188,87,Slovenia,...,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3
3,2021,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,31,1988-08-21,184,80,Poland,...,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3
4,2021,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,28,1992-02-05,175,68,Brazil,...,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3


## Load the transactions from wikipedia

**Description:** This dataset lists the top largest transactions in football

**Source:** [Wikipedia Page](https://en.wikipedia.org/wiki/List_of_most_expensive_association_football_transfers)

**Other details:** Scraped using the following tool: [wikitable2csv](https://wikitable2csv.ggor.de/)

In [6]:
transactions = pd.read_csv(
    filepath_or_buffer="BigTransactionsWikipedia.csv", 
    sep=",", 
    decimal=".")

Keep only relevant columns

In [7]:
transactions = transactions.filter(
    items=["Player","From","To","Fee(€ mln)","Year"])

Rename fields

In [8]:
transactions = transactions.rename(
    columns={"Fee(€ mln)" : "fee", "Year" : "transaction_year"})

Extract numbers from text data

In [9]:
transactions["fee"] = transactions["fee"].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="(\d+)")

In [10]:
transactions["transaction_year"] = transactions["transaction_year"].fdt.clean_text_column(
    mode="custom", 
    keep_unmatched=False, 
    regex="(\d+)")

In [11]:
transactions = transactions.astype(
    dtype={"transaction_year" : "int64",
            "fee": "int64"})

In [12]:
#hide_input
transactions.head()

,Player,From,To,fee,transaction_year
0,Neymar,Barcelona,Paris Saint-Germain,222,2017
1,Kylian Mbappé,Monaco,Paris Saint-Germain,180,2018
2,Philippe Coutinho,Liverpool,Barcelona,145,2018
3,João Félix,Benfica,Atlético Madrid,126,2019
4,Antoine Griezmann,Atlético Madrid,Barcelona,120,2019


## Use fuzzy matching to connect the data

The player names of the FIFA and the Wikipedia dataset do not match. For example, FIFA dataset uses `Neymar da Silva Santos Júnior` and the Wikipedia transaction dataset `Neymar`. To match the names that refer to the same player, fuzzy matching is used (using a library that leverages the concept of [levenstein distance](https://en.wikipedia.org/wiki/Levenshtein_distance#:~:text=Informally%2C%20the%20Levenshtein%20distance%20between,considered%20this%20distance%20in%201965.)).

To do that, we first get the unique names from the FIFA dataset in a separate dataframe.

In [13]:
player_names = pd.DataFrame({'fifa_name' : all['long_name'].unique()}).head(1000)

Pefrom fuzzy matching using a threshold of 60 (after a bit of trial and error)

In [14]:
matched_transactions = fuzzy_merge(transactions,
    right=player_names, 
    left_on="Player", 
    right_on="fifa_name", 
    threshold=60, 
    limit=1)

Transaction data with the FIFA name in a column called `matches`

In [15]:
#hide_input
matched_transactions.head()

,Player,From,To,fee,transaction_year,matches
0,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior
1,Kylian Mbappé,Monaco,Paris Saint-Germain,180,2018,Kylian Mbappé Lottin
2,Philippe Coutinho,Liverpool,Barcelona,145,2018,Philippe Coutinho Correia
3,João Félix,Benfica,Atlético Madrid,126,2019,João Félix Sequeira
4,Antoine Griezmann,Atlético Madrid,Barcelona,120,2019,Antoine Griezmann


# 2. Analysis A: Performance improvement after transaction

Assuming that the transactions were completed at a fair market value and future improvements were not priced in, the players' score improvement can be used to rank the transactions.

`score change = #score 2 years after transaction - #score transaction year`

## Get+2 year mark

Create +2y mark

In [16]:
matched_transactions["+2y"] = matched_transactions["transaction_year"].add(
    other=2)

## Get yearly player score data

Get scores per year

In [17]:
year_scores = all.filter(
    items=["year","long_name","overall"])

In [18]:
year_scores = year_scores.rename(
    columns={ "long_name" : "fifa_name", "overall" : "score"})

In [19]:
#hide_input
year_scores.head()

,year,fifa_name,score
0,2021,Lionel Andrés Messi Cuccittini,93
1,2021,Cristiano Ronaldo dos Santos Aveiro,92
2,2021,Jan Oblak,91
3,2021,Robert Lewandowski,91
4,2021,Neymar da Silva Santos Júnior,91


Rename the matched column as `fifa_name` for easier matching

In [20]:
matched_transactions = matched_transactions.rename(
    columns={ "matches" : "fifa_name"})

## Merge the yearly player score data into the transactions table

Join with the score of the year of the transaction

In [21]:
transactions_with_scores = matched_transactions.merge(
    right=year_scores, 
    how="left", 
    left_on=["transaction_year","fifa_name"], 
    right_on=["year","fifa_name"])

In [22]:
transactions_with_scores = transactions_with_scores.rename(
    columns={"year" : "year0",
             "score": "score_transaction_year"})

Join with the score of 2 years after the transaction

In [23]:
transactions_with_scores = transactions_with_scores.merge(
    right=year_scores, 
    how="inner", 
    left_on=["+2y","fifa_name"], 
    right_on=["year","fifa_name"])

In [24]:
transactions_with_scores = transactions_with_scores.rename(
    columns={"score" : "score_+2y",
             "score": "score_+2y"})

Now we have the scores at the year of the transaction and two years later

In [25]:
#hide_input
transactions_with_scores.head()

,Player,From,To,fee,transaction_year,fifa_name,+2y,year0,score_transaction_year,year,score_+2y
0,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,2017.0,92.0,2019,92
1,Kylian Mbappé,Monaco,Paris Saint-Germain,180,2018,Kylian Mbappé Lottin,2020,2018.0,83.0,2020,89
2,Philippe Coutinho,Liverpool,Barcelona,145,2018,Philippe Coutinho Correia,2020,2018.0,86.0,2020,86
3,João Félix,Benfica,Atlético Madrid,126,2019,João Félix Sequeira,2021,2019.0,69.0,2021,81
4,Antoine Griezmann,Atlético Madrid,Barcelona,120,2019,Antoine Griezmann,2021,2019.0,89.0,2021,87


## Calculate performance difference

In [26]:
transactions_with_scores = transactions_with_scores.dropna(
    axis="index", 
    how="any", 
    subset=["score_transaction_year","score_+2y"])

In [27]:
transactions_with_scores["score_delta"] = transactions_with_scores["score_+2y"].subtract(
    other=transactions_with_scores["score_transaction_year"])

In [28]:
transactions_with_scores = transactions_with_scores.sort_values(
    by="score_delta", 
    ascending=False)

In [29]:
transactions_with_scores['transaction name'] = transactions_with_scores['Player'] + '-' + transactions_with_scores['transaction_year'].apply(lambda x: str(int(x)))

In [30]:
#hide_input
transactions_with_scores.head()

,Player,From,To,fee,transaction_year,fifa_name,+2y,year0,score_transaction_year,year,score_+2y,score_delta,transaction name
3,João Félix,Benfica,Atlético Madrid,126,2019,João Félix Sequeira,2021,2019.0,69.0,2021,81,12.0,João Félix-2019
33,Alisson,Roma,Liverpool,62,2018,Alisson Ramsés Becker,2020,2018.0,80.0,2020,89,9.0,Alisson-2018
21,Frenkie de Jong,Ajax,Barcelona,75,2019,Frenkie de Jong,2021,2019.0,77.0,2021,85,8.0,Frenkie de Jong-2019
13,Virgil van Dijk,Southampton,Liverpool,84,2018,Virgil van Dijk,2020,2018.0,83.0,2020,90,7.0,Virgil van Dijk-2018
23,Kevin De Bruyne,VfL Wolfsburg,Manchester City,75,2015,Kevin De Bruyne,2017,2015.0,81.0,2017,88,7.0,Kevin De Bruyne-2015


## Results

Assuming that transactions where completed at a **fair market value**, the score improvement of the players is a metric that can give us an idea of what has been the best transaction.

In [31]:
transactions_with_scores = transactions_with_scores.sort_values(
    by="score_delta", 
    ascending=True)

According to this metric, Atlético de Madrid's purchase of João Félix's in 2019 was the best performing transaction (score change from 69 to 81), followed by Liverpool's purchase of Alisson in 2018 (score change from 80 to 89).

The least performing transactions according to this metric were Real Madrid's purchase of Eden Hazard (score change from 91 to 88), Barça's Griezmann (score change from 89 to 87) and Inter's Lukaku (score change from 87 to 85).

In [32]:
#hide input
HTML(px.bar(transactions_with_scores,
    template="seaborn",
    y="transaction name", 
    x="score_delta", 
    title="Score increase after a given transaction",
    labels={"score_delta" : "Δ fifa score after 2 yers"},
    height=1000).to_html(include_plotlyjs='cdn'))

# Analysis B: Calculate performance in years after purchase

Given [countless evidence of how irrational humans are](https://en.wikipedia.org/wiki/Behavioral_economics#:~:text=Behavioral%20economics%20(also%2C%20behavioural%20economics,implied%20by%20classical%20economic%20theory.), it's fair to assume that the purchase price is not always a reasonable market price. 

In that case, how could one determine the best deals?
One potential metric is value delivery for money invested. Notice this is not only your classical value for money, given value is not a static parameter here, and it changes every year.

`avg. value delivered = SUM(value) for each year after purchase / #year`


## Match the club name

To get the performance of the players before joining a club, we need to again have the same values in the club column, i.e. match `Barcelona` with `FC Barcelona`.

In [33]:
clubs = pd.DataFrame({'club_name' : all['club_name'].unique()}).head(2000)

Match the name of the buyer club as it is in the FIFA in a new column

In [34]:
matched_transactions = fuzzy_merge(matched_transactions, clubs, 'To', 'club_name', threshold=60, limit=1)

In [35]:
matched_transactions = matched_transactions.rename(
    columns={"matches" : "buyer_club"})

In [36]:
#hide_input
matched_transactions.head()

,Player,From,To,fee,transaction_year,fifa_name,+2y,buyer_club
0,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,Paris Saint-Germain
1,Kylian Mbappé,Monaco,Paris Saint-Germain,180,2018,Kylian Mbappé Lottin,2020,Paris Saint-Germain
2,Philippe Coutinho,Liverpool,Barcelona,145,2018,Philippe Coutinho Correia,2020,FC Barcelona
3,João Félix,Benfica,Atlético Madrid,126,2019,João Félix Sequeira,2021,Atlético Madrid
4,Antoine Griezmann,Atlético Madrid,Barcelona,120,2019,Antoine Griezmann,2021,FC Barcelona


## Get yearly performance & club info

In [37]:
performance_time_club = all.filter(
    items=["year","long_name","overall","club_name","value_eur"])

In [38]:
performance_time_club = performance_time_club.rename(
    columns={"year" : "score_year", "long_name" : "fifa_name", "overall" : "score"})

In [39]:
#hide_input
performance_time_club.head()

,score_year,fifa_name,score,club_name,value_eur
0,2021,Lionel Andrés Messi Cuccittini,93,FC Barcelona,67500000
1,2021,Cristiano Ronaldo dos Santos Aveiro,92,Juventus,46000000
2,2021,Jan Oblak,91,Atlético Madrid,75000000
3,2021,Robert Lewandowski,91,FC Bayern München,80000000
4,2021,Neymar da Silva Santos Júnior,91,Paris Saint-Germain,90000000


## Merge with performance

For each of the transactions, get all the performance rows for the same player

In [40]:
matched_transactions = matched_transactions.merge(
    right=performance_time_club, 
    how="inner", 
    left_on=["fifa_name"], 
    right_on=["fifa_name"])

In [41]:
#hide_input
matched_transactions.head()

,Player,From,To,fee,transaction_year,fifa_name,+2y,buyer_club,score_year,score,club_name,value_eur
0,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,Paris Saint-Germain,2021,91,Paris Saint-Germain,90000000
1,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,Paris Saint-Germain,2020,92,Paris Saint-Germain,105500000
2,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,Paris Saint-Germain,2018,92,Paris Saint-Germain,123000000
3,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,Paris Saint-Germain,2019,92,Paris Saint-Germain,118500000
4,Neymar,Barcelona,Paris Saint-Germain,222,2017,Neymar da Silva Santos Júnior,2019,Paris Saint-Germain,2017,92,FC Barcelona,109000000


## Get relevant datapoints after the transactions 

Keep only the performance for the years **after a transaction**, i.e. the value delivered to the buying club

In [42]:
matched_transactions = matched_transactions.query("transaction_year>=2015")

In [43]:
matched_transactions['afer_purchase'] = matched_transactions.eval("buyer_club==club_name & score_year >= transaction_year")

In [44]:
matched_transactions = matched_transactions.query("afer_purchase==True", engine="python").copy()

In [45]:
matched_transactions['transaction name'] = matched_transactions['Player'] + '-' + matched_transactions['transaction_year'].apply(lambda x: str(int(x)))

## Final calculation

In [46]:
perf_after_transaction = matched_transactions.fdt.pivot_table(
    index=["transaction name","transaction_year","fee"], 
    index_type="flat", 
    aggfunc={"score" : ["sum","size"]})

In [47]:
perf_after_transaction['avg_performance'] = perf_after_transaction['score_sum']/perf_after_transaction['score_size']

In [48]:
perf_after_transaction['perf_per_eur'] = perf_after_transaction['avg_performance'] / perf_after_transaction['fee']

## Result

Sort data

In [49]:
perf_after_transaction = perf_after_transaction.sort_values(
    by="perf_per_eur", 
    ascending=True)

It is not uncommon for companies to try several "models" of the world. One of the reasons is to be able to capture different nuances and perspectives.

Sometimes, different models agree, and in this case, we see the purchase of Alisson by Liverpool in 2018 as a very well-performing transaction. But João Félix Sequeira, which we previously saw as the best transaction, did not perform that well under this metric, which helps us understand it was a good but expensive deal.

In [50]:
#hide_input
HTML(px.bar(perf_after_transaction,
    labels={"perf_per_eur" : "performance points [0-100] per mn. €"}, 
    template="seaborn", 
    title="Value for money in top transactions",
    y="transaction name", 
    x="perf_per_eur",
    height=1000).to_html(include_plotlyjs='cdn'))

## Caveats and discussion
While some players stay in the new club for several years, others may be transferred after just a couple of seasons. To control for this, we need to normalize by the number of years the player stays.

An even better metric would be the net price per total performance delivered. This metric would consider the net price, i.e., the value of the purchase minus the sale value. But given the restricted time range of the dataset (2015-2021), we have very little data about actual sales at market values and thus decided not to pursue this analysis. 